 # Jak działa i czy jest skuteczna metoda OT w etykietowaniu nieznanego i różniącego się od wejściowego i poetykietowanego zbioru danych. 


## Problem do rozwiązania
- GENERUJEMY dwa zbiory „klasyfikujące” te same obiekty, dla których zastosowano jednak inne zbiory danych. Zakładamy, że zbiór A posiada znane etykiety. Zbiór B posiada je także, lecz zakładamy, że ich nie posiada. 

- Należy dokonać transformacji A w B wykorzystując OT i przydzielić etykiety przykładom ze zbioru B (w przestrzeni wyjściowej).
## Zbiór danych
- Wybierz dwa dowolne zbiory danych A i B do klasyfikacji zawierający umiarkowaną liczbę przykładów max m1,m2=300-1000 i o stosunkowo średniej wymiarowości (ok N=10-20)
- A i B mogą mieć różną ilość przykładów m1 oraz odpowiednio m2. Niech zbiór A będzie nazywany zbiorem ZNANYM (znane są etykiety) zbiór B to zbiór NIEZNANY (etykiety nieznane)


In [11]:
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_classification
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
import ot

# Parametry zbiorów
N_FEATURES = 15
N_CLASSES = 3
M = 500

# Tworzenie zbiorów: zwiększamy separację klas
Ax, Ay = make_classification(
    n_samples=M,
    n_features=N_FEATURES,
    n_informative=12,
    n_classes=N_CLASSES,
    class_sep=2.0,  # zwiększona separacja klas
    random_state=42
)

Bx, By = make_classification(
    n_samples=M,
    n_features=N_FEATURES,
    n_informative=12,
    n_redundant=3,
    n_classes=N_CLASSES,
    class_sep=2.0,  # zwiększona separacja klas
    random_state=43
)

# Standaryzacja
scaler = StandardScaler()
Ax = scaler.fit_transform(Ax)
Bx = scaler.transform(Bx)

# Redukcja wymiarów za pomocą PCA
pca = PCA(n_components=10)
Ax_pca = pca.fit_transform(Ax)
Bx_pca = pca.transform(Bx)

# Prawdopodobieństwa równomierne
a = np.ones((M,)) / M
b = np.ones((M,)) / M

# Koszt transportu
C = ot.dist(Ax_pca, Bx_pca)
C /= C.max()  # normalizacja

# Sinkhorn z większą regularyzacją
reg = 0.1  # większa regularyzacja
T = ot.sinkhorn(a, b, C, reg)

# One-hot dla etykiet A
Y_onehot = np.zeros((M, N_CLASSES))
Y_onehot[np.arange(M), Ay] = 1

# Predykcja
B_pred_soft = T.T @ Y_onehot
B_pred = np.argmax(B_pred_soft, axis=1)

# Ocena
acc = accuracy_score(By, B_pred)
print(f"Dokładność przypisanych etykiet OT: {acc:.4f}")


Dokładność przypisanych etykiet OT: 0.4060


In [2]:

# Parametry zbiorów
N_FEATURES = 15
N_CLASSES = 3
M1 = 500  # liczba próbek w A (ZNANY)
M2 = 700  # liczba próbek w B (NIEZNANY)

# ===== ZBIÓR A: ZNANY =====
A_X, A_y = make_classification(
    n_samples=M1,
    n_features=N_FEATURES,
    n_informative=12,
    n_redundant=3,
    n_classes=N_CLASSES,
    random_state=42
)

# Standaryzacja
scaler_A = StandardScaler()
A_X = scaler_A.fit_transform(A_X)

# ===== ZBIÓR B: NIEZNANY =====
B_X, B_y = make_classification(
    n_samples=M2,
    n_features=N_FEATURES,
    n_informative=12,
    n_redundant=3,
    n_classes=N_CLASSES,
    random_state=999  # inny seed => inny rozkład
)

# Standaryzacja + transformacja PCA + dodanie szumu
scaler_B = StandardScaler()
B_X = scaler_B.fit_transform(B_X)

pca = PCA(n_components=N_FEATURES)
B_X = pca.fit_transform(B_X)

# Losowa rotacja
rotation = np.linalg.qr(np.random.randn(N_FEATURES, N_FEATURES))[0]
B_X = B_X @ rotation

# Dodaj lekki szum
B_X += np.random.normal(0, 0.1, B_X.shape)

# Finalne kształty
print(f"Zbiór A: {A_X.shape}, etykiety: {np.unique(A_y)}")
print(f"Zbiór B: {B_X.shape}, etykiety: {np.unique(B_y)} (tylko do ewaluacji)")

Zbiór A: (500, 15), etykiety: [0 1 2]
Zbiór B: (700, 15), etykiety: [0 1 2] (tylko do ewaluacji)


## Zadanie 2

Dokonaj transformacji PCA w zbiorze A i w zbiorze B. W każdym z nich przetransformuj przykłady do przestrzeni 2D (maksymalna wariancja) i zwizualizuj. To są twoje empiryczne rozkłady na których będziesz pracować. 


## Zadanie 3

Zaznajom się z problematyką Optimal Transport (np. wykład,  optimal-transport-basics | Kaggle etc.) oraz (dla chętnych!) z blogiem A simple introduction on Sinkhorn distances | by Jianfeng Wang | Medium.
W jaki sposób definiuje się wektor wag dla rozkładu eksperymentalnego? Załóż że wagi pochodzą z rozkładu przy pomocy którego wygenerowane zostały dane.


## Zadanie 4

Ponieważ A jest ZNANYM zbiorem dokonaj transformacji A w B zgodnie z przykładem: Optimal Transport between 2D empirical distributions — POT Python Optimal Transport 0.8.2 documentation (pythonot.github.io). 


## Zadanie 5
- sróbować zrobić to dla m1!=m2 (preferowałbym zarówno m1>m2 i m2<m1).
- Na podstawie wyników transformacji określ nieznane etykiety zbioru B i oceń dokładność tej aproksymacji.
- Sformułuj wnioski.
